Importing libraries and defining column names which are expected in target table (because we have multiple tables and no other unique identifier found)

In [239]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

target_column1 = "% of Total Holdings"
target_column2 = "% of Total Holding"
selected_df = None


Defining a function to extract fund name and fund ID from url

In [240]:
import re

def extract_fund_details(url):
    # Pattern to match the fund name and fund ID
    pattern = r"mutual-funds/(.*?)/portfolio-overview/(.*)"
    
    # Extract fund name and fund ID using regex
    match = re.search(pattern, url)
    if match:
        fund_name = match.group(1)
        fund_id = match.group(2)
        return fund_name, fund_id
    else:
        return None, None
    

##Defining a function to get the web scrapping data and extract relevant table

In [241]:
def get_mf_data(tab_name, urls):

    response_dfs = []

    for url in urls:
        fund_name, fund_id = extract_fund_details(url)
        #print(f"Fund name {fund_name}")
        response = requests.get(url)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            tables = soup.find_all('table', class_='mctable1')
            for idx, table in enumerate(tables):
                data = []
                rows = table.find_all('tr')
                header = [ele.text.strip() for ele in rows[0].find_all('th')]
                for row in rows[1:]:
                    cols = row.find_all('td')
                    cols = [ele.text.strip() for ele in cols]
                    data.append(cols)
                max_len = max(len(row) for row in data + [header])
                data = [row + [''] * (max_len - len(row)) for row in data]
                df = pd.DataFrame(data, columns=header)
                df['Fund Name'] = fund_name
                df['Fund Id'] = fund_id
                df['MF ACF'] = "Equity"
                df['MF Type'] = tab_name


                if (target_column1 in df.columns or target_column2 in df.columns) and len(df.index) > 10:
                    if response_dfs:
                        if response_dfs[-1].columns.equals(df.columns):
                            response_dfs.append(df)
                            break
                    else:
                        response_dfs.append(df)
                        break

        else:
            print('Failed to fetch data for URL:', url)

    if response_dfs:
        combined_df = pd.concat(response_dfs, ignore_index=True)
    else:
        print("No data found.")
    
    file_path_to_Save  = "C:\\Users\\Monil\\OneDrive\\Desktop\\Prabha Analytics\\Data\\" + tab_name + ".csv"
    combined_df.to_csv(file_path_to_Save, index=False)
    print(f"File saved at {file_path_to_Save}")
    return(combined_df)


# Equity ACF (1/2)

In [ ]:
file_path = "C:\\Users\\Monil\\OneDrive\\Desktop\\MSDS\\05_DA_MSDS_460\\99_Project\\Data\\MF_Data\\equity_mf_url_part1.xlsx"

xl        = pd.ExcelFile(file_path)

# Get the list of tab names
tab_names = xl.sheet_names

# Create a dictionary to store tab names and URLs
tab_urls = {}

# Iterate over each tab
for tab_name in tab_names:
    # Read the tab as a DataFrame
    df = xl.parse(tab_name)
    # Store the URLs from the 'Portfolio Link' column
    urls = df['Portfolio Link'].tolist()
    # Add the tab name and URLs to the dictionary
    tab_urls[tab_name] = urls

dfs = []

for tab_name, urls in tab_urls.items():
    print(f"Tab Name: {tab_name}")
    # Assuming get_mf_data returns a DataFrame
    df = get_mf_data(tab_name, urls)
    dfs.append(df)

# Concatenate all DataFrames in the list
all_equity_mf_1 = pd.concat(dfs, ignore_index=True)
all_equity_mf_1.to_csv("C:\\Users\\Monil\\OneDrive\\Desktop\\MSDS\\05_DA_MSDS_460\\99_Project\\Data\\MF_Data\\Results\\all_equity_mf_1.csv", index = False)


Tab Name: 01_value
Failed to fetch data for URL: https://www.moneycontrol.com/mutual-funds/quantum-long-term-equity-value-fund-direct-plan-growth/portfolio-overview/MQU001
File saved at C:\Users\Monil\OneDrive\Desktop\Prabha Analytics\Data\01_value.csv
Tab Name: 02_small_cap
File saved at C:\Users\Monil\OneDrive\Desktop\Prabha Analytics\Data\02_small_cap.csv
Tab Name: 03_large_mid
Failed to fetch data for URL: https://www.moneycontrol.com/mutual-funds/bajaj-finserv-large-and-mid-cap-fund-direct-plan-growth/portfolio-overview/MBFA048
File saved at C:\Users\Monil\OneDrive\Desktop\Prabha Analytics\Data\03_large_mid.csv
Tab Name: 04_elss
Failed to fetch data for URL: https://www.moneycontrol.com/mutual-funds/sundaram-long-term-micro-cap-tax-advantage-fund-series-iv-direct-plan-growth/portfolio-overview/MSN1290
File saved at C:\Users\Monil\OneDrive\Desktop\Prabha Analytics\Data\04_elss.csv
Tab Name: 05_contra
File saved at C:\Users\Monil\OneDrive\Desktop\Prabha Analytics\Data\05_contra.csv


# Equity ACF (2/2)

In [246]:
file_path = "C:\\Users\\Monil\\OneDrive\\Desktop\\MSDS\\05_DA_MSDS_460\\99_Project\\Data\\MF_Data\\equity_mf_url_part2.xlsx"
xl        = pd.ExcelFile(file_path)

# Get the list of tab names
tab_names = xl.sheet_names

# Create a dictionary to store tab names and URLs
tab_urls = {}

# Iterate over each tab
for tab_name in tab_names:
    # Read the tab as a DataFrame
    df = xl.parse(tab_name)
    # Store the URLs from the 'Portfolio Link' column
    urls = df['Portfolio Link'].tolist()
    # Add the tab name and URLs to the dictionary
    tab_urls[tab_name] = urls

dfs = []

for tab_name, urls in tab_urls.items():
    print(f"Tab Name: {tab_name}")
    # Assuming get_mf_data returns a DataFrame
    df = get_mf_data(tab_name, urls)
    dfs.append(df)

# Concatenate all DataFrames in the list
all_equity_mf_2 = pd.concat(dfs, ignore_index=True)
all_equity_mf_2.to_csv("C:\\Users\\Monil\\OneDrive\\Desktop\\MSDS\\05_DA_MSDS_460\\99_Project\\Data\\MF_Data\\Results\\all_equity_mf_2.csv", index = False)

Tab Name: 07_multi
File saved at C:\Users\Monil\OneDrive\Desktop\Prabha Analytics\Data\07_multi.csv
Tab Name: 08_mid
File saved at C:\Users\Monil\OneDrive\Desktop\Prabha Analytics\Data\08_mid.csv
Tab Name: 09_div_yield
File saved at C:\Users\Monil\OneDrive\Desktop\Prabha Analytics\Data\09_div_yield.csv
Tab Name: 10_focus
File saved at C:\Users\Monil\OneDrive\Desktop\Prabha Analytics\Data\10_focus.csv
Tab Name: 11_large
File saved at C:\Users\Monil\OneDrive\Desktop\Prabha Analytics\Data\11_large.csv
Tab Name: 12_sectoral
File saved at C:\Users\Monil\OneDrive\Desktop\Prabha Analytics\Data\12_sectoral.csv


# Hybrid Fund Data

In [237]:
file_path = "C:\\Users\\Monil\\OneDrive\\Desktop\\MSDS\\05_DA_MSDS_460\\99_Project\\Data\\MF_Data\\hybrid_mf_url_v2.xlsx"
xl        = pd.ExcelFile(file_path)

# Get the list of tab names
tab_names = xl.sheet_names

# Create a dictionary to store tab names and URLs
tab_urls = {}

# Iterate over each tab
for tab_name in tab_names:
    # Read the tab as a DataFrame
    df = xl.parse(tab_name)
    # Store the URLs from the 'Portfolio Link' column
    urls = df['Portfolio Link'].tolist()
    # Add the tab name and URLs to the dictionary
    tab_urls[tab_name] = urls

dfs = []

for tab_name, urls in tab_urls.items():
    print(f"Tab Name: {tab_name}")
    # Assuming get_mf_data returns a DataFrame
    df = get_mf_data(tab_name, urls)
    dfs.append(df)

# Concatenate all DataFrames in the list
hybrid_equity_mf = pd.concat(dfs, ignore_index=True)
hybrid_equity_mf.to_csv("C:\\Users\\Monil\\OneDrive\\Desktop\\MSDS\\05_DA_MSDS_460\\99_Project\\Data\\MF_Data\\Results\\all_hybrid_equity_mf.csv", index = False)

Tab Name: 01_aggressive_hybrid
File saved at C:\Users\Monil\OneDrive\Desktop\Prabha Analytics\Data\01_aggressive_hybrid.csv
Tab Name: 02_Equity_saving
Failed to fetch data for URL: https://www.moneycontrol.com/mutual-funds/pgim-india-equity-savings-fund-direct-plan-growth/portfolio-overview/MDE551
Failed to fetch data for URL: https://www.moneycontrol.com/mutual-funds/edelweiss-equity-savings-fund-direct-plan-growth/portfolio-overview/MJP305
File saved at C:\Users\Monil\OneDrive\Desktop\Prabha Analytics\Data\02_Equity_saving.csv
Tab Name: 03_conservative_hybrid
File saved at C:\Users\Monil\OneDrive\Desktop\Prabha Analytics\Data\03_conservative_hybrid.csv
Tab Name: 04_dynamic_AA
Failed to fetch data for URL: https://www.moneycontrol.com/mutual-funds/bajaj-finserv-balanced-advantage-direct-plan-growth/portfolio-overview/MBFA043
Failed to fetch data for URL: https://www.moneycontrol.com/mutual-funds/shriram-balanced-advantage-fund-direct-plan-growth/portfolio-overview/MSR015
File saved at